In [2]:
from tokenized_dataset import load_tokenized_dataset
import numpy as np

def build_masks(x, y): # x: seq_len, y: seq_len
    # Mask padded x tokens
    x_mask = np.ones((x.shape[0], x.shape[0]))
    x_pad_mask = np.where(x != 0, np.ones((x.shape[0])), 0)
    x_mask = np.multiply(np.multiply(x_mask, x_pad_mask), x_pad_mask[:, None])

    # Mask padded y tokens + add "autoregressive" mask
    y_pad_mask = np.where(y != 0, np.ones((y.shape[0])), 0)
    y_mask = np.tri(y.shape[0], y.shape[0])
    y_mask = np.multiply(np.multiply(y_mask, y_pad_mask), y_pad_mask[:, None])

    # Mask padded yx tokens
    yx_mask = np.ones((y.shape[0], x.shape[0]))
    yx_mask = np.multiply(np.multiply(yx_mask, x_pad_mask), y_pad_mask[:, None])

    return x_mask, y_mask, yx_mask

pad_and_trunc = lambda toks, seq_len: (toks + [0] * (seq_len - len(toks)))[:seq_len]
pack_batch = lambda batch: [np.stack(el) for el in map(list, zip(*batch))]

# Complete batch to batch_size with pad tokens only sequences 
def complete_last_batch(batch, batch_size):
    for _ in range(batch_size - len(batch)):
        x = np.zeros(seq_len)
        y = np.zeros(seq_len)
        x_mask, y_mask, yx_mask = build_masks(x,y)
        batch.append((x, y, x_mask, y_mask, yx_mask))
    return batch
    

def get_batched_examples(ds, batch_size, seq_len, split="train", skip_n_rows=None):
    ds_split = ds[split].skip(skip_n_rows) if skip_n_rows is not None else ds[split]

    batch = []
    for item in ds_split:
        x = np.array(pad_and_trunc(item['x'], seq_len))
        y = np.array(pad_and_trunc(item['y'], seq_len))
        x_mask, y_mask, yx_mask = build_masks(x,y)
        batch.append((x, y, x_mask, y_mask, yx_mask))
        if len(batch) == batch_size:
            yield pack_batch(batch)
            batch = []
    if split!="train" and len(batch) > 0: # Note I don't use last few rows left in train split..
        yield pack_batch(complete_last_batch(batch, batch_size))

ds, _ = load_tokenized_dataset()

x, y, x_mask, y_mask, yx_mask = next(get_batched_examples(ds, 2, 10))
print(x)
print(y)

Loading dataset
Loading tokenizer bpe_tokenizer_ds_train_all_merges_35k.pickle
Tokenizing dataset
[[23954 12114  3613  3586 12466     0     0     0     0     0]
 [ 3667 21876 28867  3586 12466  3613  3586  3960  4795  3751]]
[[31352  3606 29833     0     0     0     0     0     0     0]
 [ 3638 37931  3610  3979 36357  3787 15348  9617 19088 27915]]


In [3]:
from tokenized_dataset import load_tokenized_dataset
import numpy as np

def get_batched_examples_packed(ds, batch_size, seq_len, start_tok, end_tok, pack_frac = 0.5, split="train", skip_n_rows=None):
    assert split=="train"
    ds_split = ds[split].skip(skip_n_rows) if skip_n_rows is not None else ds[split]
    
    batch = []
    for item in ds_split:
        # Either append to previous batch item or create new one
        if len(batch)>0 and (len(batch[-1][0]) < seq_len * pack_frac and len(batch[-1][1]) < seq_len * pack_frac):
            pack_func = lambda x, y: x + [end_tok, start_tok] + y
            batch[-1] = (pack_func(batch[-1][0], item['x']), pack_func(batch[-1][1], item['y']))
        else:
            batch.append((item['x'],item['y']))
            
        if len(batch) == batch_size:
            def convert_batch_item(x, y):
                x = np.array(pad_and_trunc(x, seq_len))
                y = np.array(pad_and_trunc(y, seq_len))
                x_mask, y_mask, yx_mask = build_masks(x,y)
                return (x, y, x_mask, y_mask, yx_mask)
            batch = [convert_batch_item(*it) for it in batch]
            print(len(batch[0][0]))
            print(len(batch[1][0]))
            
            yield pack_batch(batch)
            batch = []
            
    if split!="train" and len(batch) > 0: # Note I don't use last few rows left in train split..
        yield pack_batch(complete_last_batch(batch, batch_size))

ds, (_, _, tokenizer_vocab_size) = load_tokenized_dataset()
START_TOK = tokenizer_vocab_size + 1
END_TOK = tokenizer_vocab_size + 2

x, y, x_mask, y_mask, yx_mask = next(get_batched_examples_packed(ds, 10, 10, START_TOK, END_TOK, pack_frac=0.75))
print(x)
print(y)

Loading dataset
Loading tokenizer bpe_tokenizer_ds_train_all_merges_35k.pickle
Tokenizing dataset
10
10
[[23954 12114  3613  3586 12466 38560 38559  3667 21876 28867]
 [ 3585 15064  3626  3829  3850  3827 31442  3586  4489 11945]
 [ 3829  3827 15275  3612  6044  3599  3745  7010  3594  3586]
 [ 3594  3586 32712  3667  4156  4239  3617 20984  3612  6958]
 [ 5895  4535  3788 11576  3687  3745  6958  3655  4435  3571]
 [ 9339  5544 19234  3614 23218  3612  6958  3655  4435  3571]
 [ 6698  4468  3599  3612  5318  3613 16215 38560 38559  3829]
 [ 3943  3613  3586  4641 31869  4017  4043  9155  3594 22023]
 [ 4081  3632  3743  7996  3687  8240  6698  4468  3617 12907]
 [14445  4148  4031 11341  3667  7401  3642  6866  3613  3586]]
[[31352  3606 29833 38560 38559  3638 37931  3610  3979 36357]
 [ 3869  3725 26275 24456  3732  3606  3619 23742  7960    35]
 [ 3755  4794  6502  3606 10816  4052  3884  7320  3755 16943]
 [ 4910  4547  3638  3725 17257  3762  3700  3732  3837  3606]
 [ 3638  5897

In [33]:
def build_masks(x, y, x_packs=None, y_packs=None, yx_packs=None): # x: seq_len, y: seq_len
    # Mask padded x tokens
    x_mask = np.ones((x.shape[0], x.shape[0]))
    x_pad_mask = np.where(x != 0, np.ones((x.shape[0])), 0)
    x_mask = np.multiply(np.multiply(x_mask, x_pad_mask), x_pad_mask[:, None])
    if x_packs is not None:
        x_mask = np.multiply(x_mask, x_packs)

    # Mask padded y tokens + add "autoregressive" mask
    y_pad_mask = np.where(y != 0, np.ones((y.shape[0])), 0)
    y_mask = np.tri(y.shape[0], y.shape[0])
    y_mask = np.multiply(np.multiply(y_mask, y_pad_mask), y_pad_mask[:, None])
    if y_packs is not None:
        y_mask = np.multiply(y_mask, y_packs)

    # Mask padded yx tokens
    yx_mask = np.ones((y.shape[0], x.shape[0]))
    yx_mask = np.multiply(np.multiply(yx_mask, x_pad_mask), y_pad_mask[:, None])
    if yx_packs is not None:
        yx_mask = np.multiply(yx_mask, yx_packs)

    return x_mask, y_mask, yx_mask

def ones_block_diag(lens_dim0, lens_dim1):
    result = np.zeros((sum(lens_dim0), sum(lens_dim1)))
    start_ind_dim0 = 0
    start_ind_dim1 = 0
    for len_dim0, len_dim1 in zip(lens_dim0, lens_dim1):
        result[start_ind_dim0:start_ind_dim0+len_dim0, start_ind_dim1:start_ind_dim1+len_dim1] = np.ones((len_dim0, len_dim1))
        start_ind_dim0 +=len_dim0
        start_ind_dim1 +=len_dim1
    return result

def create_packs(x_lens, y_lens):
    x_packs = ones_block_diag(x_lens, x_lens)
    y_packs = ones_block_diag(y_lens, y_lens)
    yx_packs = ones_block_diag(y_lens, x_lens)
    return x_packs, y_packs, yx_packs


x = np.array([1, 2, 3, 5, 6, 7, 0])
y = np.array([11, 12, 13, 15, 16, 0, 0])
print("INPUT")
print(x)
print(y)

print("\nPACKS")
x_packs, y_packs, yx_packs = create_packs([3, 3, 1], [3, 2, 2])
print(x_packs)
print(y_packs)
print(yx_packs)

print("\nMASKS")
x_mask, y_mask, yx_mask = build_masks(x, y, x_packs=x_packs, y_packs=y_packs, yx_packs = yx_packs)
print(x_mask)
print(y_mask)
print(yx_mask)

INPUT
[1 2 3 5 6 7 0]
[11 12 13 15 16  0  0]

PACKS
[[1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]]
[[1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 1. 1.]]
[[1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]]

MASKS
[[1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0.]


In [52]:
import jax.numpy as jnp
from jax import random

def pos_encodings(x, indices): # input: seq_len x emb_dim
    seq_len, emb_dim = x.shape

    #indices = jnp.arange(seq_len)[:, None]
    print(indices.shape)
    div_term = jnp.fromfunction(lambda i: 1 / pow(10000, 2 * i/emb_dim), (int(emb_dim/2),), dtype=float)[None, :]
    print(div_term.shape)

    print(f'\nPOS_ARRAY')
    pos_array = jnp.dot(indices, div_term)
    print(pos_array.shape)
    print(pos_array)

    print(f'\nSTACKED')
    stacked = jnp.stack((jnp.sin(pos_array), jnp.cos(pos_array)), axis=2)
    print(stacked.shape)
    print(stacked)
    return stacked.reshape(seq_len, emb_dim)

x =random.uniform(random.PRNGKey(0), (7, 4)) # 7 seq_len, 4 emb_dim
#x_indices = jnp.arange(x.shape[0])[:, None]
# TODO: indices need to be inferred from mask OR 
# should we get lens from data collator instead,
# (creating indices from lens is relatively straightforward)
# However, lens have variable size, which doesn't make it perfect
# for JAX
x_indices = jnp.array([0, 1, 2, 0, 1, 2, 3])[:, None] 
print(x_indices)
print(pos_encodings(x, x_indices))

#y =random.uniform(random.PRNGKey(0), (7, 4)) # 7 seq_len, 4 emb_dim
#print(pos_encodings(y))

[[0]
 [1]
 [2]
 [0]
 [1]
 [2]
 [3]]
(7, 1)
(1, 2)

POS_ARRAY
(7, 2)
[[0.   0.  ]
 [1.   0.01]
 [2.   0.02]
 [0.   0.  ]
 [1.   0.01]
 [2.   0.02]
 [3.   0.03]]

STACKED
(7, 2, 2)
[[[ 0.          1.        ]
  [ 0.          1.        ]]

 [[ 0.841471    0.5403023 ]
  [ 0.00999983  0.99995   ]]

 [[ 0.90929747 -0.4161468 ]
  [ 0.01999867  0.9998    ]]

 [[ 0.          1.        ]
  [ 0.          1.        ]]

 [[ 0.841471    0.5403023 ]
  [ 0.00999983  0.99995   ]]

 [[ 0.90929747 -0.4161468 ]
  [ 0.01999867  0.9998    ]]

 [[ 0.14112    -0.9899925 ]
  [ 0.0299955   0.99955004]]]
[[ 0.          1.          0.          1.        ]
 [ 0.841471    0.5403023   0.00999983  0.99995   ]
 [ 0.90929747 -0.4161468   0.01999867  0.9998    ]
 [ 0.          1.          0.          1.        ]
 [ 0.841471    0.5403023   0.00999983  0.99995   ]
 [ 0.90929747 -0.4161468   0.01999867  0.9998    ]
 [ 0.14112    -0.9899925   0.0299955   0.99955004]]
